In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow
import statsmodels.api as sm
from CTL.causal_tree_learn import CausalTree
from fancyimpute import IterativeImputer as MICE
import graphviz
import re
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import scipy
import econml
from econml.metalearners import SLearner, TLearner, XLearner
from econml.inference import BootstrapInference
from bartpy.sklearnmodel import SklearnModel # package maybe problematic
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import pickle

import warnings
warnings.filterwarnings("ignore")

/Users/fionaguo/.pyenv/versions/3.7.8/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Using TensorFlow backend.


# Average Treatment Effect

In [6]:
with open('sharp_outcomes.pickle', 'rb') as handle:
    outcome_vars = pickle.load(handle)
    
df_raw = pd.read_csv("sharp_np_q_fitness_R_processed.csv")
df_raw.rename(columns={"Group_np": "Group", "GoodVO2_np": "GoodVO2", "Unnamed: 0":"SubjectID"},inplace=True)
df_raw.set_index("SubjectID",inplace=True)
Group = df_raw["Group"].copy()
GoodVO2 = df_raw["GoodVO2"].copy()
df_raw.drop(["Group","GoodVO2"],axis=1,inplace=True)

In [ ]:
changeRelationalMemory = outcome_vars

In [12]:
# z-test to see if these distributions are significantly different - left-tailed
mean = np.zeros((4,8))
std = np.zeros((4,8))
for i in range(4): # each group
    for ((k,v),j) in zip(outcome_vars.items(),range(8)): # each outcome
        mean[i,j] = v[Group.index[Group==(i+1)]].mean()
        std[i,j] = v[Group.index[Group==(i+1)]].std()

N_G1 = len(Group[Group==1])
N_G2 = len(Group[Group==2])
N_G3 = len(Group[Group==3])
N_G4 = len(Group[Group==4])

zscore = pd.DataFrame(np.zeros((5,8)),index=["G2-G1","G3-G1","G4-G1","G3-G2","G4-G3"],columns=list(outcome_vars.keys()))

for i in range(8): # each outcome
    # G2-G1
    zscore.iloc[0,i] = (mean[1,i] - mean[0,i]) / np.sqrt(np.square(std[1,i])/N_G2 + np.square(std[0,i])/N_G1)
    # G3-G1
    zscore.iloc[1,i] = (mean[2,i] - mean[0,i]) / np.sqrt(np.square(std[2,i])/N_G3 + np.square(std[0,i])/N_G1)
    # G4-G1
    zscore.iloc[2,i] = (mean[3,i] - mean[0,i]) / np.sqrt(np.square(std[3,i])/N_G4 + np.square(std[0,i])/N_G1)
    # G3-G2
    zscore.iloc[3,i] = (mean[2,i] - mean[1,i]) / np.sqrt(np.square(std[2,i])/N_G3 + np.square(std[1,i])/N_G2)
    # G4-G3
    zscore.iloc[4,i] = (mean[3,i] - mean[2,i]) / np.sqrt(np.square(std[3,i])/N_G4 + np.square(std[2,i])/N_G3)

# pval = pd.DataFrame(np.zeros((5,4)),index=["G2-G1","G3-G1","G4-G1","G3-G2","G4-G3"],columns=["changeRelationalMemory","changeProcessingSpeed","changeDecisionMaking","changeDSSTCorrect"])
p_values = scipy.stats.norm.sf(abs(zscore.values))
p_values = pd.DataFrame(p_values,index=["G2-G1","G3-G1","G4-G1","G3-G2","G4-G3"],columns=list(outcome_vars.keys())) 
mean = pd.DataFrame(mean,index=["G1","G2","G3","G4"],columns=list(outcome_vars.keys()))
std = pd.DataFrame(std,index=["G1","G2","G3","G4"],columns=list(outcome_vars.keys()))
print("p values")
p_values


p values


,changeRelationalMemory,changeProcessingSpeed,changeDecisionMaking,changeDSSTCorrect,changeVO2,changeWHRatio,changeFatPerc,changeFlexibility
G2-G1,0.061193,0.119713,0.000409,0.343729,3.583011e-10,0.172346,0.000052,0.000037
G3-G1,0.001633,0.205904,0.000236,0.333872,6.104349e-07,0.085159,0.009237,0.004173
G4-G1,0.185841,0.137836,0.001305,0.174667,1.175835e-05,0.456548,0.013439,0.000135
G3-G2,0.124289,0.317008,0.396099,0.499847,7.179998e-02,0.337425,0.009946,0.094462
G4-G3,0.026783,0.370471,0.328686,0.283336,2.779153e-01,0.073759,0.406521,0.129124


In [14]:
mean

,changeRelationalMemory,changeProcessingSpeed,changeDecisionMaking,changeDSSTCorrect,changeVO2,changeWHRatio,changeFatPerc,changeFlexibility
G1,0.238739,0.137869,-0.426511,-0.079309,-0.595317,0.114325,0.362831,-0.421416
G2,-0.056991,-0.093773,0.154795,-0.001900,0.441964,-0.070808,-0.418474,0.252067
G3,-0.256609,-0.008341,0.198600,-0.001832,0.180436,-0.139737,0.002745,0.026159
G4,0.070563,-0.063078,0.121054,0.095687,0.079884,0.093258,-0.034192,0.230682


In [15]:
# ate
ate = pd.DataFrame(np.zeros((5,8)),index=["G2-G1","G3-G1","G4-G1","G3-G2","G4-G3"],columns=list(outcome_vars.keys()))
for i in range(8):
    ate.iloc[0,i] = mean.iloc[1,i] - mean.iloc[0,i]
    ate.iloc[1,i] = mean.iloc[2,i] - mean.iloc[0,i]
    ate.iloc[2,i] = mean.iloc[3,i] - mean.iloc[0,i]
    ate.iloc[3,i] = mean.iloc[2,i] - mean.iloc[1,i]
    ate.iloc[4,i] = mean.iloc[3,i] - mean.iloc[2,i]
ate


,changeRelationalMemory,changeProcessingSpeed,changeDecisionMaking,changeDSSTCorrect,changeVO2,changeWHRatio,changeFatPerc,changeFlexibility
G2-G1,-0.295730,-0.231642,0.581305,0.077409,1.037281,-0.185133,-0.781305,0.673483
G3-G1,-0.495349,-0.146210,0.625111,0.077477,0.775753,-0.254063,-0.360086,0.447575
G4-G1,-0.168176,-0.200947,0.547565,0.174997,0.675201,-0.021068,-0.397023,0.652098
G3-G2,-0.199619,0.085432,0.043805,0.000068,-0.261527,-0.068930,0.421218,-0.225908
G4-G3,0.327173,-0.054737,-0.077546,0.097520,-0.100552,0.232995,-0.036937,0.204523


In [17]:
# save results
ate_dict = {"ate":ate,"mean":mean,"pval":p_values}

with open("sharp_ate.pickle","wb") as handle:
    pickle.dump(ate_dict,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
with open("sharp_ate.pickle","rb") as handle:
    ate_dict = pickle.load(handle)